In [1]:
# Install the latest version of the OpenAI package
!pip install --upgrade openai

# Import necessary libraries
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

# Load environment variables from a .env file
load_dotenv(find_dotenv())


True

In [2]:
# Initialize the OpenAI client with the API key
api_key = 'sk-proj-9Gw8RhdBekIhLh05pGrKT3BlbkFJRhweDVVTVC9C85RryRBJ'
client = OpenAI(api_key=api_key)


In [3]:
# Define tools for the assistant
tools = [
    {
        "function": {
            "name": "get_weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": [
                            "c",
                            "f"
                        ],
                        "description": "The temperature unit to use. Infer this from the user's location."
                    }
                },
                "required": ["location"]
            },
            "description": "Get the current weather"
        },
        "type": "function"
    }
]


In [4]:
# Create a new assistant
weather_helper = client.beta.assistants.create(
    model='gpt-4',
    name='AK-Functions-Demo-From-Notebook',
    instructions="You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    tools=tools,
    description='This is an assistant created from a notebook.'
)
print(weather_helper.model_dump_json(indent=2))
#"id": "asst_fSdkp728yrq2bqwC8Ti0BoNh"

{
  "id": "asst_fSdkp728yrq2bqwC8Ti0BoNh",
  "created_at": 1722403062,
  "description": "This is an assistant created from a notebook.",
  "instructions": "You are a helpful assistant. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
  "metadata": {},
  "model": "gpt-4",
  "name": "AK-Functions-Demo-From-Notebook",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "get_weather",
        "description": "Get the current weather",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": [
                "c",
                "f"
              ],
              "description": "The temperature unit to use. Infer this from the user's location."
         

In [96]:
# Create a new thread
thread = client.beta.threads.create()
print(thread)


Thread(id='thread_iUrMIlAwdlyQlWFPuzsniOPa', created_at=1722411053, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [97]:
# Predefined weather data
weather_data = {
    'New York': 25,
    'Mumbai': 34,
    'Jaipur': 21,
    'Manali': 5
}


In [98]:
user_input = input("Please enter your question: ")

Please enter your question: what is the weather in mumbai ?


In [100]:
print(user_input)

what is the weather in mumbai ?


In [101]:


# Create a user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_input
)
print(message)


Message(id='msg_w83DumbgonL2V29NrQH5WQSP', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='what is the weather in mumbai ?'), type='text')], created_at=1722411099, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_iUrMIlAwdlyQlWFPuzsniOPa')


In [102]:
# Create a run for the assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id='asst_fSdkp728yrq2bqwC8Ti0BoNh',
    instructions="Please address the user as Akshat."
)
print(run.status)


queued


In [103]:
# Retrieve the run details to get the tool_call_id
retrieve_run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(retrieve_run.status)


requires_action


In [104]:
# Check the status of the run and only proceed if it's not complete
if retrieve_run.status != 'complete':
    # List the thread messages
    thread_messages = client.beta.threads.messages.list(thread.id)
    
    user_messages = ''
    assistant_messages = ''

    for tm in reversed(thread_messages.data):
        content = tm.content[0].text.value
        if tm.role == 'user':
            user_messages += content + '\n'
        if tm.role == 'assistant':
            assistant_messages += content + '\n'

    print("User's query:")
    print(user_messages.strip())

    print("Assistant's response:")
    print(assistant_messages.strip())

    submit_tool_outputs = retrieve_run.required_action.submit_tool_outputs
    temp = json.loads(submit_tool_outputs.tool_calls[0].function.arguments)

    weather_output = f"The weather for {temp.get('location')} is {weather_data.get(temp.get('location'))} celsius"
    print(weather_output)

    tool_call_id = submit_tool_outputs.tool_calls[0].id

    # Submit the tool output
    client.beta.threads.runs.submit_tool_outputs(
        run_id=run.id,
        thread_id=thread.id,
        tool_outputs=[
            {
                "tool_call_id": tool_call_id,
                "output": weather_output,
            }
        ]
    )
else:
    print("The run is already complete, no need to find the weather again.")


User's query:
what is the weather in mumbai ?
what is the weather in mumbai ?
Assistant's response:

The weather for Mumbai is 34 celsius
